In [1]:
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import lxml
import re
from random import seed
from collections import Counter
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

C:\Users\MoJo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LinkOutdoorWomens4x100m = "http://www.ustfccca.org/records-lists/outdoor-collegiate-all-time-bests?gender=2&sport=2&div=20527&top=25&event=18"

In [3]:
page = requests.get(LinkOutdoorWomens4x100m)
soup = BeautifulSoup(page.text,'html')
soup

table = soup.find("table")

In [4]:
Header = soup.findAll('th')
Headers = []
for h in Header:
    Headers.append(h.text)

In [5]:
Headers

['Team', 'Institution', 'Mark', 'Date', '']

In [6]:
row_soup = soup.findAll('tr')

In [7]:
row_soup

[<tr><th style="width: 35%;">Team</th><th colspan="2" style="width: 30%;">Institution</th><th style="width: 25%;">Mark</th><th style="width: 10%;">Date</th><th></th></tr>,
 <tr><td colspan="100" style="font-weight: bolder;">UNOFFICIAL Top 25 Performers and Performances</td></tr>,
 <tr><td colspan="5"><div class="container-table" style="width: 100%;"><table class="embed" width="100%"><thead><tr></tr></thead><tr><td style="width: 35%;"><span>(1) Texas</span></td><td style="width: 5%;"><span class="group-logo-inline"><img alt="Texas" class="group-logo-inline" src="/images/logos/rankings-article/texas.png" title="Texas"/></span></td><td style="width: 25%;"> </td><td style="width: 25%;"><a href="https://www.tfrrs.org/results/81300/4938111" target="_blank">41.55</a> <span class="tooltip" style="font-size: smaller;">q<span class="tooltiptext" style="font-size: smaller;">q = Performance occurred during a non-final round.</span></span> </td><td style="width: 10%;text-align:right;" title="NCAA D

In [8]:
rows = []
for row in row_soup:
    row_data = []
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    rows.append(row_data)

In [9]:
rows

[[],
 ['UNOFFICIAL Top 25 Performers and Performances'],
 ['(1) Texas\xa041.55 qq = Performance occurred during a non-final round. 6/8/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '(1) Texas',
  '',
  '\xa0',
  '41.55 qq = Performance occurred during a non-final round. ',
  '6/8/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 [],
 ['(1) Texas',
  '',
  '\xa0',
  '41.55 qq = Performance occurred during a non-final round. ',
  '6/8/2023'],
 ['\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 ['\xa0\xa0\xa0\xa0— TEXAS {2}\xa041.60 6/10/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '\xa0\xa0\xa0\xa0— TEXAS {2}',
  '',
  '\xa0',
  '41.60 ',
  '6/10/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 [],
 ['\xa0\xa0\xa0\xa0— TEXAS {2}', '', '\xa0', '41.60 ', '6/10/2023'],
 ['\xa0\xa0\xa0\xa0\x

In [10]:
len(rows)

1464

In [11]:
rows[1446]

['\xa0\xa0\xa0\xa0— SOUTHERN CALIFORNIA {35}\xa043.39 qq = Performance occurred during a non-final round. 6/9/2016\xa0\xa0\xa0\xa0\xa0Destinee Brown, Deanna Hill, Alexis Faulknor, Tynia Gaither',
 '\xa0\xa0\xa0\xa0— SOUTHERN CALIFORNIA {35}',
 '',
 '\xa0',
 '43.39 qq = Performance occurred during a non-final round. ',
 '6/9/2016',
 '\xa0\xa0\xa0\xa0\xa0Destinee Brown, Deanna Hill, Alexis Faulknor, Tynia Gaither']

In [12]:
len(rows)

1464

In [13]:
RealRows = []

In [14]:
for i in range(0,1447):
    if len(rows[i]) > 1:
        RealRows.append(rows[i])
        

In [15]:
RealRows

[['(1) Texas\xa041.55 qq = Performance occurred during a non-final round. 6/8/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '(1) Texas',
  '',
  '\xa0',
  '41.55 qq = Performance occurred during a non-final round. ',
  '6/8/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 ['(1) Texas',
  '',
  '\xa0',
  '41.55 qq = Performance occurred during a non-final round. ',
  '6/8/2023'],
 ['\xa0\xa0\xa0\xa0— TEXAS {2}\xa041.60 6/10/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '\xa0\xa0\xa0\xa0— TEXAS {2}',
  '',
  '\xa0',
  '41.60 ',
  '6/10/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 ['\xa0\xa0\xa0\xa0— TEXAS {2}', '', '\xa0', '41.60 ', '6/10/2023'],
 ['\xa0\xa0\xa0\xa0— TEXAS {3}\xa041.89📺 5/12/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '\xa0\xa0\xa0\xa0— TEXAS {3}',
  '',
  '

In [16]:
RealRealRows = RealRows[::2]
RealRealRows

[['(1) Texas\xa041.55 qq = Performance occurred during a non-final round. 6/8/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '(1) Texas',
  '',
  '\xa0',
  '41.55 qq = Performance occurred during a non-final round. ',
  '6/8/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 ['\xa0\xa0\xa0\xa0— TEXAS {2}\xa041.60 6/10/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '\xa0\xa0\xa0\xa0— TEXAS {2}',
  '',
  '\xa0',
  '41.60 ',
  '6/10/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 ['\xa0\xa0\xa0\xa0— TEXAS {3}\xa041.89📺 5/12/2023\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS',
  '\xa0\xa0\xa0\xa0— TEXAS {3}',
  '',
  '\xa0',
  '41.89📺 ',
  '5/12/2023',
  '\xa0\xa0\xa0\xa0\xa0Julien ALFRED, Ezinne ABBA, Rhasidat ADELEKE, Kevona DAVIS'],
 ['\xa0\xa0\xa0\xa0— TEXAS {4}\xa042.00📺 4/1/2023\xa0\xa0\x

In [17]:
RealRealRows[1][4]

'41.60 '

In [18]:
len(RealRealRows)

372

In [19]:
OfficialTimes = []

In [20]:
for i in range(0,len(RealRealRows)):
    OfficialTimes.append(RealRealRows[i][4])
OfficialTimes

['41.55 qq = Performance occurred during a non-final round. ',
 '41.60 ',
 '41.89📺 ',
 '42.00📺 ',
 '42.05 ',
 '42.09 qq = Performance occurred during a non-final round. ',
 '42.12 ',
 '42.12 ',
 '42.14 ',
 '42.15 qq = Performance occurred during a non-final round. ',
 '42.21 ',
 '42.25 ',
 '42.29 ',
 '42.30 qq = Performance occurred during a non-final round. ',
 '42.30 ',
 '42.32 qq = Performance occurred during a non-final round. ',
 '42.34 qq = Performance occurred during a non-final round. ',
 '42.34 ',
 '42.35 AA = Performance occurred at altitude (>1000 meters above sea level) ',
 '42.35 ',
 '42.36  ',
 '42.37 ',
 '42.38 ',
 '42.42 ',
 '42.42 ',
 '42.42 qq = Performance occurred during a non-final round. ',
 '42.44 qq = Performance occurred during a non-final round. ',
 '42.45 ',
 '42.46 ',
 '42.46 ',
 '42.49 qq = Performance occurred during a non-final round. ',
 '42.49 ',
 '42.49 qq = Performance occurred during a non-final round. ',
 '42.50 ',
 '42.50 AA = Performance occurred 

In [21]:
RealRealRows[0][1]

'(1) Texas'

In [22]:
Names = []

for i in range(0,len(RealRealRows)):
    Names.append(RealRealRows[i][1])
Names

['(1) Texas',
 '\xa0\xa0\xa0\xa0— TEXAS {2}',
 '\xa0\xa0\xa0\xa0— TEXAS {3}',
 '\xa0\xa0\xa0\xa0— TEXAS {4}',
 '(2) LSU',
 '\xa0\xa0\xa0\xa0— LSU {2}',
 '\xa0\xa0\xa0\xa0— LSU {3}',
 '(3) Oregon',
 '\xa0\xa0\xa0\xa0— LSU {4}',
 '\xa0\xa0\xa0\xa0— TEXAS {5}',
 '(4) Southern California',
 '\xa0\xa0\xa0\xa0— LSU {5}',
 '\xa0\xa0\xa0\xa0— LSU {6}',
 '(5) Kentucky',
 '\xa0\xa0\xa0\xa0— KENTUCKY {2}',
 '\xa0\xa0\xa0\xa0— LSU {7}',
 '\xa0\xa0\xa0\xa0— TEXAS {6}',
 '\xa0\xa0\xa0\xa0— OREGON {2}',
 '\xa0\xa0\xa0\xa0— TEXAS {7}',
 '\xa0\xa0\xa0\xa0— OREGON {3}',
 '(6) Texas A&M',
 '\xa0\xa0\xa0\xa0— TEXAS {8}',
 '\xa0\xa0\xa0\xa0— TEXAS {9}',
 '\xa0\xa0\xa0\xa0— TEXAS {10}',
 '\xa0\xa0\xa0\xa0— TEXAS {11}',
 '\xa0\xa0\xa0\xa0— LSU {8}',
 '\xa0\xa0\xa0\xa0— SOUTHERN CALIFORNIA {2}',
 '\xa0\xa0\xa0\xa0— SOUTHERN CALIFORNIA {3}',
 '\xa0\xa0\xa0\xa0— KENTUCKY {3}',
 '\xa0\xa0\xa0\xa0— KENTUCKY {4}',
 '\xa0\xa0\xa0\xa0— KENTUCKY {5}',
 '\xa0\xa0\xa0\xa0— LSU {9}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {2}',


In [23]:
OfficialReGexNames = []

In [24]:
for i in range(0,len(RealRealRows)):
    if Names[i].startswith("\xa0\xa0"):
        OfficialReGexNames.append(Names[i][4:])
    else:
        OfficialReGexNames.append(Names[i])
        
OfficialReGexNames

['(1) Texas',
 '— TEXAS {2}',
 '— TEXAS {3}',
 '— TEXAS {4}',
 '(2) LSU',
 '— LSU {2}',
 '— LSU {3}',
 '(3) Oregon',
 '— LSU {4}',
 '— TEXAS {5}',
 '(4) Southern California',
 '— LSU {5}',
 '— LSU {6}',
 '(5) Kentucky',
 '— KENTUCKY {2}',
 '— LSU {7}',
 '— TEXAS {6}',
 '— OREGON {2}',
 '— TEXAS {7}',
 '— OREGON {3}',
 '(6) Texas A&M',
 '— TEXAS {8}',
 '— TEXAS {9}',
 '— TEXAS {10}',
 '— TEXAS {11}',
 '— LSU {8}',
 '— SOUTHERN CALIFORNIA {2}',
 '— SOUTHERN CALIFORNIA {3}',
 '— KENTUCKY {3}',
 '— KENTUCKY {4}',
 '— KENTUCKY {5}',
 '— LSU {9}',
 '— TEXAS A&M {2}',
 '— LSU {10}',
 '— LSU {11}',
 '— KENTUCKY {6}',
 '— LSU {12}',
 '— LSU {13}',
 '— KENTUCKY {7}',
 '— SOUTHERN CALIFORNIA {4}',
 '— KENTUCKY {8}',
 '— LSU {14}',
 '— LSU {15}',
 '(7) Alabama',
 '— LSU {16}',
 '— TEXAS A&M {3}',
 '— TEXAS A&M {4}',
 '— SOUTHERN CALIFORNIA {5}',
 '— OREGON {4}',
 '— LSU {17}',
 '— LSU {18}',
 '— TEXAS A&M {5}',
 '— LSU {19}',
 '— TEXAS A&M {6}',
 '— LSU {20}',
 '— LSU {21}',
 '— KENTUCKY {9}',
 '—

In [25]:
len(OfficialTimes)

372

In [26]:
len(OfficialReGexNames)

372

In [27]:
CleanedNames = []

for i in range(0,len(RealRealRows)):
    if OfficialReGexNames[i].startswith("—"):
        CleanedNames.append(OfficialReGexNames[i][2:])
    if OfficialReGexNames[i].startswith("("):
        CleanedNames.append(OfficialReGexNames[i][4:])        
CleanedNames

['Texas',
 'TEXAS {2}',
 'TEXAS {3}',
 'TEXAS {4}',
 'LSU',
 'LSU {2}',
 'LSU {3}',
 'Oregon',
 'LSU {4}',
 'TEXAS {5}',
 'Southern California',
 'LSU {5}',
 'LSU {6}',
 'Kentucky',
 'KENTUCKY {2}',
 'LSU {7}',
 'TEXAS {6}',
 'OREGON {2}',
 'TEXAS {7}',
 'OREGON {3}',
 'Texas A&M',
 'TEXAS {8}',
 'TEXAS {9}',
 'TEXAS {10}',
 'TEXAS {11}',
 'LSU {8}',
 'SOUTHERN CALIFORNIA {2}',
 'SOUTHERN CALIFORNIA {3}',
 'KENTUCKY {3}',
 'KENTUCKY {4}',
 'KENTUCKY {5}',
 'LSU {9}',
 'TEXAS A&M {2}',
 'LSU {10}',
 'LSU {11}',
 'KENTUCKY {6}',
 'LSU {12}',
 'LSU {13}',
 'KENTUCKY {7}',
 'SOUTHERN CALIFORNIA {4}',
 'KENTUCKY {8}',
 'LSU {14}',
 'LSU {15}',
 'Alabama',
 'LSU {16}',
 'TEXAS A&M {3}',
 'TEXAS A&M {4}',
 'SOUTHERN CALIFORNIA {5}',
 'OREGON {4}',
 'LSU {17}',
 'LSU {18}',
 'TEXAS A&M {5}',
 'LSU {19}',
 'TEXAS A&M {6}',
 'LSU {20}',
 'LSU {21}',
 'KENTUCKY {9}',
 'SOUTHERN CALIFORNIA {6}',
 'LSU {22}',
 'SOUTHERN CALIFORNIA {7}',
 'LSU {23}',
 'OREGON {5}',
 'LSU {24}',
 'Arkansas',
 'LSU {2

In [28]:
CleanedNames2 = []

for i in range(0,len(RealRealRows)):
    if CleanedNames[i].startswith(" "):
        CleanedNames2.append(CleanedNames[i][1:])
    else:
        CleanedNames2.append(CleanedNames[i])        
CleanedNames2

['Texas',
 'TEXAS {2}',
 'TEXAS {3}',
 'TEXAS {4}',
 'LSU',
 'LSU {2}',
 'LSU {3}',
 'Oregon',
 'LSU {4}',
 'TEXAS {5}',
 'Southern California',
 'LSU {5}',
 'LSU {6}',
 'Kentucky',
 'KENTUCKY {2}',
 'LSU {7}',
 'TEXAS {6}',
 'OREGON {2}',
 'TEXAS {7}',
 'OREGON {3}',
 'Texas A&M',
 'TEXAS {8}',
 'TEXAS {9}',
 'TEXAS {10}',
 'TEXAS {11}',
 'LSU {8}',
 'SOUTHERN CALIFORNIA {2}',
 'SOUTHERN CALIFORNIA {3}',
 'KENTUCKY {3}',
 'KENTUCKY {4}',
 'KENTUCKY {5}',
 'LSU {9}',
 'TEXAS A&M {2}',
 'LSU {10}',
 'LSU {11}',
 'KENTUCKY {6}',
 'LSU {12}',
 'LSU {13}',
 'KENTUCKY {7}',
 'SOUTHERN CALIFORNIA {4}',
 'KENTUCKY {8}',
 'LSU {14}',
 'LSU {15}',
 'Alabama',
 'LSU {16}',
 'TEXAS A&M {3}',
 'TEXAS A&M {4}',
 'SOUTHERN CALIFORNIA {5}',
 'OREGON {4}',
 'LSU {17}',
 'LSU {18}',
 'TEXAS A&M {5}',
 'LSU {19}',
 'TEXAS A&M {6}',
 'LSU {20}',
 'LSU {21}',
 'KENTUCKY {9}',
 'SOUTHERN CALIFORNIA {6}',
 'LSU {22}',
 'SOUTHERN CALIFORNIA {7}',
 'LSU {23}',
 'OREGON {5}',
 'LSU {24}',
 'Arkansas',
 'LSU {2

In [29]:
Times = []
for i in range(0,len(OfficialTimes)):
    Times.append(OfficialTimes[i][:5])
    
Times

['41.55',
 '41.60',
 '41.89',
 '42.00',
 '42.05',
 '42.09',
 '42.12',
 '42.12',
 '42.14',
 '42.15',
 '42.21',
 '42.25',
 '42.29',
 '42.30',
 '42.30',
 '42.32',
 '42.34',
 '42.34',
 '42.35',
 '42.35',
 '42.36',
 '42.37',
 '42.38',
 '42.42',
 '42.42',
 '42.42',
 '42.44',
 '42.45',
 '42.46',
 '42.46',
 '42.49',
 '42.49',
 '42.49',
 '42.50',
 '42.50',
 '42.51',
 '42.52',
 '42.52',
 '42.53',
 '42.53',
 '42.55',
 '42.55',
 '42.56',
 '42.56',
 '42.56',
 '42.56',
 '42.56',
 '42.57',
 '42.59',
 '42.59',
 '42.59',
 '42.59',
 '42.59',
 '42.61',
 '42.61',
 '42.62',
 '42.63',
 '42.63',
 '42.63',
 '42.63',
 '42.63',
 '42.64',
 '42.64',
 '42.65',
 '42.65',
 '42.65',
 '42.66',
 '42.68',
 '42.68',
 '42.69',
 '42.69',
 '42.70',
 '42.70',
 '42.71',
 '42.72',
 '42.73',
 '42.74',
 '42.75',
 '42.76',
 '42.76',
 '42.77',
 '42.77',
 '42.78',
 '42.79',
 '42.79',
 '42.80',
 '42.80',
 '42.80',
 '42.81',
 '42.81',
 '42.82',
 '42.82',
 '42.82',
 '42.82',
 '42.82',
 '42.83',
 '42.83',
 '42.83',
 '42.83',
 '42.84',


In [30]:
Womens4x100mDF = pd.DataFrame({'Names': CleanedNames2, 'OfficialTimes': Times})
Womens4x100mDF.to_csv('Womens4x100m.csv', index=False)